# Introducción a los dibujitos

Esta serie de notebooks es para aprender a crear imagenes con los paquetes del lenguaje de programacion Julia que ayuden a visualizar experimentos computacionales de carácter matemático, en particular en dinámica holomorfa.

Vamos a estar utilizando los paquetes ``Images`` y ``Colors`` para la creacion y manipulación de imagenes. También usaremos ``FileIO`` para manipulación de archivos: cargar y guardar imagenes.  Otros paquetes que usaremos son ``Distributions`` para generar numeros aleatorios de manera sencilla. El primer bloque de codigo es para agregar estos paquetes a nuestra instalación de Julia y solo es necesario correrlo una vez.

In [ ]:
Pkg.update()
Pkg.add("Images")
Pkg.add("Colors")
Pkg.add("FileIO")
Pkg.add("Distributions")

El siguiente bloque es para cargar los paquetes instalados en a sesión actual de Julia y debe de correrse en cada ocasión que se reinicie el notebook.

In [ ]:
using Images, Colors

## ¿Qué es un dibujito?

Un *dibujito* es un arreglo bidimensional de pixeles que representa un muestreo de una region del plano complejo. 
El color que se le asigna a cada pixel tendrá algun significado que le querramos dar.

En los ejemplos que veremos a continuación, la región que estaremos representando es un rectángulo. La siguiente función genera un arreglo bidimensional de números complejos que es una muestra equidistribuida de un cuadrado centrado en ``centro`` y que contiene un disco de un ``radio`` dado. La resolución de la muestra esta dada por los parametros ``alto`` y ``ancho``.

In [ ]:
function muestra(radio=2.0, centro=0.0im, alto=1000, ancho=1000)
    dx=(2.0*radio)/(1.0*ancho)  #el ancho que representa un pixel
    dy=(2.0im*radio)/(1.0*alto) #el alto que representa un pixel
    z0 = centro + radio*(1.0im - 1.0) #el primer valor de la muestra es la esquina superior izquierda
    return [z0 + (x*dx) - (y*dy) for y in 0:alto-1, x in 0:ancho-1] 
end

En este caso incluimos valores por omisión para la función `muestra`, estos son los valores que aparecen en el primer renglon.

`function muestra(radio=2.0, centro=0.0im, alto=1000, ancho=1000)`

Al incluir valores por omisión siempre los ubicamos al final de la lista de parámetros (en este ejemplo todos los parametros tienen valors por omisión). 

En este ejemplo podemos llamar a la función `muestra` con 4, 3, 2, 1 o ningun parametro, al hacerlo se le asignaran los valores que proporcinemos a las variables en el orden de aparición. Por ejemplo, `muestra(1.0, 1.0im)` creara una muestra con los parametros `radio = 1.0`, `centro = 1.0im` y los valores por omisión `alto=1000, ancho=1000`.


Veamos un ejemplo del arreglo de numeros que nos entrega la función ``muestra``:

In [ ]:
muestra(3.0, 0.0, 5, 5)

## Mi primer dibujito

Como un primer ejemplo de dibujito vamos a pintar el argumento de los numeros complejos.
Este no es un dibujito dinamico, pero nos da una idea de como funciona el muestreo.
Para esto usaremos el espacio de color HSV (Tono, Saturacion, Brillo). 
- El Tono (Hue) es un valor entre 0.0 y 360.0, y recorre todo el espectro de color en un circulo.
- La Saturación es un valor entre 0.0 y 1.0
- El Brillo (Value) es un valor entre 0.0 y 1.0

In [ ]:
function argumento_complejo(Z)
    HSV{Float64}(mod(360.0 + rad2deg(angle(Z)), 360.0),1.0,1.0)
end

La función ``argumento_complejo`` tiene como parametro un numero complejo y nos regresa un color cuyo tono depende del argumento complejo y con saturación y brillo totales. Vamos a aplicar la función a todo el muestreo usando *broadcasting*, es decir, vamos a aplicar la función a un arreglo de numeros entrada-a-entrada, lo que nos da un arreglo de colores (pixeles) que Julia mismo intepreta como imagen y el notebook lo muestra en pantalla.

In [ ]:
argumento_complejo.(muestra(1.0, 0.0, 500, 500))

## Mi segundo dibujito

Ahora vamos a pintar la *gráfica* de una función de variable compleja. De hecho, vamos a pintar el argumento de la imagen.

In [ ]:
function grafica_compleja(F, radio=2.0, centro=0.0im, alto=1000, ancho=1000)
    broadcast(Z -> argumento_complejo(F(Z)), muestra(radio, centro, alto, ancho))
end

La función anterior ``grafica_compleja`` toma como parametro una función $F: \mathbb{C} \rightarrow \mathbb{C}$ y los parametros de la muestra, y regresa un arreglo de colores (imagen) que representa el argumento complejo de $F(z)$ para los $z$ en la muestra.

Veamos algunos ejemplos. En estos ejemplos usaremos la notacion ``->`` para denotar funciones, asi como la declaracion explicita de funciones y el uso de funciones existentes en Julia.

In [ ]:
grafica_compleja(Z -> Z^3)

In [ ]:
mi_polinomio(Z) = Z^3 + (0.48735im - 0.49864)*Z + (0.3198541 + 0.67546im)
grafica_compleja(mi_polinomio)

In [ ]:
grafica_compleja(Z -> sin(Z), 10.0) #ventana de radio 10.0

## Mas dibujitos

Estas *graficas complejas* pueden adaptarse a nuestros gustos y necesidades. Tenemos la libertad de *pintar* el contradominio de nuestras funciones de la manera en que querramos y la *grafica compleja* correspondiente será entonces un *pullback* del espacio de colores.

En este ejemplo pintaremos solo en blanco y negro, dependiendo del signo de la parte imaginaria de los numeros.

In [ ]:
function signo_imaginario(Z)
    if imag(Z) > 0
        return HSV{Float64}(0.0, 0.0, 0.0) #negro
    else
        return HSV{Float64}(0.0, 0.0, 1.0) #blanco
    end
end

function nueva_grafica_compleja(F, color, radio=2.0, centro=0.0im, alto=1000, ancho=1000)
    broadcast(Z -> color(F(Z)), muestra(radio, centro, alto, ancho))
end

La funcion `nueva_grafica_compleja` toma como parámetros la función que vamos a graficar, la función de color a la que haremos *pullback* y los parámetros de ventana.

In [ ]:
nueva_grafica_compleja(z -> z^3, signo_imaginario)

In [ ]:
nueva_grafica_compleja(mi_polinomio, signo_imaginario)

In [ ]:
nueva_grafica_compleja(x -> sin(x), signo_imaginario, 10.0)

En Julia podemos promediar colores si estan en formato RGB:

In [ ]:
function promedio_color(A)
    mean([convert(RGB, x) for x in A])
end

nueva_grafica_compleja(z -> z^3, p -> promedio_color([signo_imaginario(p), argumento_complejo(p)]))

In [ ]:
nueva_grafica_compleja(mi_polinomio, p -> promedio_color([signo_imaginario(p), argumento_complejo(p)]))

In [ ]:
nueva_grafica_compleja(z -> sin(z), p -> promedio_color([signo_imaginario(p), argumento_complejo(p)]), 10.0)